**Neighborhood Growth Model**

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
import category_encoders as ce
import xgboost as xgb
import matplotlib.pyplot as mpl


### NEED TO CREATE A FRAMEWORK TO STANDARDIZE HOW MANY COLUMNS THE ENCODING NEEDS

In [2]:
# load all data
raw_data_t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")
raw_data_t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1995.csv")

# load the first quadrat that we will be working with
quad1_t1 = raw_data_t1.loc[raw_data_t1['quadrat']<=50]
quad1_t2 = raw_data_t2.loc[raw_data_t2['quadrat']<=50]

# we only care about the tree IDs and DBHs of quad1_t2
expected_labels = quad1_t2[['treeID', 'dbh']]
expected_labels = expected_labels.rename(columns={"dbh": "dbh2", "treeID": "treeID2"})

#quad1_t1.head()
# simplify the data to have less features
quad1_t1 = quad1_t1[['treeID', 'sp', 'gx', 'gy', 'dbh']]
quad1_t1 = quad1_t1.rename(columns={"dbh": "dbh1", "treeID":"treeID1"})

encoder= ce.BinaryEncoder(cols=['sp'],return_df=True)
quad1_t1 = encoder.fit_transform(quad1_t1)
df_combined = pd.concat([quad1_t1, expected_labels], axis=1)

# Drop rows with any NaN values
df_combined_clean = df_combined.dropna()

# round each dbh to the nearest 5!!
# featurest1[:,10] = np.around(featurest1[:,10]/5)* 5

# Separate the cleaned DataFrame and labels
df_clean = df_combined_clean[quad1_t1.columns]
labels_clean = df_combined_clean[expected_labels.columns]

quad1_t1 = df_clean.to_numpy()
expected_labels = labels_clean.to_numpy()

quad1_t1 = quad1_t1.astype(np.float32)
expected_labels = expected_labels.astype(np.float32)

/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_640/2288554603.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")
/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_640/2288554603.py:3: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1995.csv")


In [3]:
# HOW MANY COLS TO ENCODE SPECIES?
a = 8

In [4]:
ids = quad1_t1[:,0]
# X WOULD BE a +1, y would be a+2
x_coordinates = quad1_t1[:, a+1]  
y_coordinates = quad1_t1[:, a+2]
coord_matrix = np.column_stack((x_coordinates, y_coordinates))
spatial_tree = sp.spatial.KDTree(coord_matrix)

In [5]:
# would be a below
nn_dist_matrix2 = np.zeros((len(coord_matrix),6))
nn_ind_matrix2 = np.zeros((len(coord_matrix),6))
nn_feats = np.column_stack((quad1_t1[:, 0:a+1], quad1_t1[:,a+3]))
feats_matrix = np.zeros((len(coord_matrix),(6*(a+2))))

In [6]:
# NEW CODE AS OF JULY 6 - THIS WORKS!!!!!!!!
for i, tree in enumerate(coord_matrix):
    dist2, ind2 = spatial_tree.query(tree, k=6)
    nn_ind_matrix2[i] = ids[ind2]
    nn_dist_matrix2[i]= dist2

    nn_row = nn_feats[i].reshape(1,a+2)
    inc = 0
    for j in nn_ind_matrix2[i][1:]:
        row_ind = np.where(quad1_t1[:,0] == j)

        real_row = (quad1_t1[row_ind])
        distance = dist2[1:][inc].reshape(1,1)
        dbh = real_row[:,a+3].reshape(1,1)
        nn_row = np.hstack((nn_row, distance, real_row[:,1:a+1],dbh))
        inc += 1
    
    feats_matrix[i] = nn_row
    
nn_ind_matrix2 = nn_ind_matrix2[:,1:]
nn_dist_matrix2 = nn_dist_matrix2[:,1:]
# remove the columns with the focal tree's OWN distance and IDS



In [7]:
# change the labels so that they show GROWTH, not future DBH - better for readability of loss
expected_labels[:,1] = expected_labels[:,1] - quad1_t1[:,a+3]
expected_labels = np.where (expected_labels<0, 0, expected_labels)
X_train, X_test, y_train, y_test = train_test_split(feats_matrix, expected_labels, test_size=0.3)

feats = X_train[:,1:]
labels = y_train[:,1]

test_ids = X_test[:,0]
test_feats = X_test[:,1:]

Using SKLearn's Random Forest Regressor

In [12]:
rf = RandomForestRegressor(max_depth=6, n_estimators=100)
rf.fit(feats,labels)

preds = rf.predict(test_feats)
preds_matrix = np.column_stack((test_ids, preds))

error1 = sklearn.metrics.mean_squared_error(y_test[:,1], preds)
print (np.sqrt(error1))
error1a = r2_score(y_test[:,1], preds)
print (error1a)

11.167602224639062
0.04647977121182889


Using XGBoost Regressor

In [9]:
xgb_tree = xgb.XGBRegressor()
xgb_tree.fit(feats,labels)
preds2 = xgb_tree.predict(test_feats)
predictions_matrix = np.column_stack((test_ids, preds2))
error2 = sklearn.metrics.mean_squared_error(y_test[:,1], preds2)
print (np.sqrt(error2))
error2a = r2_score (y_test[:,1], preds2)
print (error2a)

11.23569
0.034817317400235304


In [13]:
cv = KFold(n_splits=5)
scores = cross_validate(rf, quad1_t1[:,1:],expected_labels[:,1], scoring=('neg_root_mean_squared_error','r2'), cv=cv)

print (scores['test_neg_root_mean_squared_error'])
print (scores['test_r2'])

[-10.1002466   -5.63812705 -25.44887027  -8.47231649 -19.06206055]
[ 0.01772266  0.14698852 -5.04979049 -0.37882216  0.02933778]


**Randomization attempts below!**

In [14]:
# generate 20 random sequences of 1 to 5 - COMPLETE
rng = np.random.default_rng()
matrix_order = np.zeros((20,5))

shuffle_this = np.array([0, 1, 2, 3, 4])
for i in range(20):
  current_row = rng.permutation(shuffle_this)
  matrix_order[i] = current_row


In [15]:
nn_dist_matrix3 = np.zeros((len(coord_matrix),6))
nn_ind_matrix3 = np.zeros((len(coord_matrix),6))
nn_feats2 = np.column_stack((quad1_t1[:, 0:a+1], quad1_t1[:,a+3]))
#feats_matrix = np.zeros((len(coord_matrix),54))

In [16]:
# Start with feats_matrix above. Then create similar matrices for each of the above
feats_matrix_big = feats_matrix.copy()
# feats matrix small is miossing focal dbh

# of columns should be a*6

zer_indices = range(a+2,2*a+4)
one_indices = range (2*a+4,3*a+6)
two_indices = range (3*a+6,4*a+8)
three_indices = range (4*a+8,5*a+10)
four_indices = range (5*a+10,6*a+12)

# the long number passing into range is ust n=20 here, but this would make it less hard-coded in
for i in range(np.shape(matrix_order)[0]):
    # loop through each permutation.
    feats_matrix_small = feats_matrix.copy()
    perm_indices = [range(a+2)]
    current_perm = matrix_order[i]
    for x in current_perm:
        if x == 0:
          perm_indices.append(zer_indices)
        elif x == 1:
           perm_indices.append(one_indices)
        elif x == 2:
           perm_indices.append(two_indices)
        elif x == 3:
           perm_indices.append(three_indices)
        elif x == 4:
           perm_indices.append(four_indices)
    flattened = []
    for sublist in perm_indices:
       for item in sublist:
          flattened.append(item)
    feats_matrix_small = feats_matrix_small[:,flattened]
    feats_matrix_big = np.vstack((feats_matrix_big,feats_matrix_small))

In [17]:
# Create the corresponding y labels - THIS WORKS! # of rows = 138x20
expected_labels2 = expected_labels.copy()
for i in range(20):
    expected_labels2 = np.vstack((expected_labels2, expected_labels))

In [18]:
cv2 = KFold(n_splits=5)
cv_rf2 = RandomForestRegressor(max_depth=8)

scores = cross_validate(cv_rf2, feats_matrix_big[:,1:], expected_labels2[:,1], scoring=('neg_root_mean_squared_error','r2'), cv=cv)
print(scores)

{'fit_time': array([37.01697111, 43.03562021, 40.67548513, 37.09245896, 43.14177704]), 'score_time': array([0.11007786, 0.12534285, 0.11203003, 0.11108303, 0.11647511]), 'test_neg_root_mean_squared_error': array([-7.73608462, -8.02349681, -8.00131782, -7.81267053, -8.49282321]), 'test_r2': array([0.56001399, 0.5160398 , 0.52980099, 0.54312353, 0.51666179])}


Create framework to print anything: (incorporated from diagnostics file)

In [28]:
names = encoder.inverse_transform(quad1_t1)
sp_list = names.sp.unique()
sp_freq = names.sp.value_counts(sort=True).index.tolist()
sp_num = sp_list.shape[0]
sp_mat = np.hstack((np.atleast_2d(sp_list).transpose(), (np.zeros((sp_num, a)))))

all_dbhs = np.arange(10,501,5).transpose()
all_dbhs_big = np.arange(10,501,5).transpose()
list_of_all_sp = []

for i in range(sp_num):
    # 1) look at initial dataset and find the tree with that ID, then find its encoding
    # find tree ID of one instance of a species
    tree_id = names[names.sp == sp_freq[i]].treeID1
    one_tree_id = tree_id.iloc[0]
    encoding = feats_matrix_big[feats_matrix_big[:,0]==one_tree_id][0,1:a+1]

    sp_label = np.atleast_2d(encoding)
    sp_repeated = np.repeat(sp_label,99,axis=0)
    sp_i = np.column_stack((sp_repeated,all_dbhs))
    all_dbhs_big = np.vstack((all_dbhs_big, all_dbhs))
    list_of_all_sp.append(sp_i)

one_big_array = np.vstack(list_of_all_sp)
all_dbhs_big = (all_dbhs_big.flatten())[:a*3234] # can be represented by a*3234?

list_of_all_preds = []
for sp in list_of_all_sp:
    # adjust as if it had all zero neighbors
    sp_adjusted = np.column_stack((sp,np.zeros((99,5*(a+2)))))
    preds = rf.predict(sp_adjusted)
    list_of_all_preds.append(preds)


Below, compare the model with 0 neighbors to the intrinsic model

In [44]:
# data as if each focal tree had no neighbors, but the actual trees are from our data

comparison = np.hstack((nn_feats,np.zeros((np.shape(nn_feats)[0],a*5))))
feats_matrix_big2 = comparison.copy()
for i in range(np.shape(matrix_order)[0]):
    # loop through each permutation.
    feats_matrix_big2 = np.vstack((feats_matrix_big2,comparison))

In [45]:
scores = cross_validate(cv_rf2, feats_matrix_big2[:,1:], expected_labels2[:,1], scoring=('neg_root_mean_squared_error','r2'), cv=cv)

In [46]:
print(scores)
# These are the r2 scores for cross validation for a model trained on 50 quadrats.
# It is only taking into account dbh and species of the FOCAL tree
# But it is organized in the format of the neighborhood model.
# Thus, we can compare it to the intrinsic model (in diagnostics2). 
# (Though the diagnostics model was trained on ALL data)

# Indeed, it is odd that the average r2 here is about 0.57. On the other hand, the r2 
# for the intrinsic model is only about 0.14 on average. They should be more comparable?

{'fit_time': array([5.19270301, 5.93953824, 5.28836203, 4.73146391, 4.69043088]), 'score_time': array([0.11352801, 0.11210704, 0.11695671, 0.10414004, 0.11284518]), 'test_neg_root_mean_squared_error': array([-7.52899542, -7.48939002, -7.61969456, -7.54150771, -7.89951323]), 'test_r2': array([0.5832549 , 0.57832761, 0.5735837 , 0.57428777, 0.58183505])}
